In [1]:
import cv2
import numpy as np 
import time
from utils import *

from keras.models import Model
from numpy import array
from keras.models import load_model
import PIL
from PIL import Image
import Utils_SRGAN, Utils_model_SRGAN
from Utils_model_SRGAN import VGG_LOSS
from scipy.spatial.distance import pdist, squareform

Using TensorFlow backend.


## SRGAN

In [2]:
def load_VGG_SRGAN():
    model_dir = './model-weights/gen_model3000.h5'
    image_shape = (96, 96, 3)
    
    loss = VGG_LOSS(image_shape=image_shape)  
    model = load_model(model_dir , custom_objects={'vgg_loss': loss.vgg_loss})
    
    return model

def test_model_for_lr_images(input_low_res, model):

    x_test_lr = Utils_SRGAN.LOAD_DATA_TEST(input_low_res)
    output_high_res = Utils_SRGAN.plot_test_generated_images(model, x_test_lr)
    return output_high_res

## Functions used to detect faces in an image
**- load_yolo_face** loads the config and weight files for Yolo-v3 trained for Face Detection

**- face_detection** detects faces and stores them in 'image_data/'

In [3]:
def load_yolo_face():
    
    model_cfg = './cfg/yolov3-face.cfg'
    model_weights = './model-weights/yolov3-wider_16000.weights'
    
    net = cv2.dnn.readNetFromDarknet(model_cfg, model_weights)
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
    
    return net

def face_detection(net, frame, current_frame, current_person, model_vgg):

        # Create a 4D blob from a frame.
        blob = cv2.dnn.blobFromImage(frame, 1 / 255, (IMG_WIDTH, IMG_HEIGHT), [0, 0, 0], 1, crop=False)

        # Sets the input to the network
        net.setInput(blob)

        # Runs the forward pass to get output of the output layers
        outs = net.forward(get_outputs_names(net))

        # Remove the bounding boxes with low confidence
        faces = post_process(frame, outs, CONF_THRESHOLD, NMS_THRESHOLD)
        print('[i] ==> # detected faces: {}'.format(len(faces)))
        print('#' * 60)

        # initialize the set of information we'll displaying on the frame
        info = [
            ('number of faces detected', '{}'.format(len(faces)))
        ]
        
        # to crop out multiple faces in the image
        for f in faces:
            x, y, w, h = f

            sub_face = frame[y+2:y+h-2, x+2:x+w-2]
            final_img = test_model_for_lr_images(sub_face, model_vgg)
            #Saves image of the current frame in jpg file
            name='./image_data/frame'+str(current_frame) + '_p'+str(current_person)+'.jpg'
            print('Creating...'+name)
            cv2.imwrite(name,final_img)
            #To prevent duplicate images
            current_person+=1

## Functions used to detect objects in an image
**- load_yolo** is used to load the config and weight files used to detect objects using COCO dataset

**- start_webcam** is used to start the webcam using OpenCV

**- detect_objects** is used to detect different objects inn an image

**- get_box_dimensions** is used bounding box, scores and class of the detected object

**- draw_labels** is used to draw labels above the bounding box of the detected object

**- webcam_detect** is used to call all the above functions to perform the whole process

In [7]:
#Load yolo
def load_yolo():
    net = cv2.dnn.readNet("model-weights/yolov3.weights", "cfg/yolov3.cfg")
    classes = []
    with open("cfg/coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    layers_names = net.getLayerNames()
    output_layers = [layers_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    return net, classes, colors, output_layers


def detect_objects(img, net, outputLayers):
    blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(outputLayers)
    return blob, outputs

def get_box_dimensions(outputs, height, width):
    boxes = []
    confs = []
    class_ids = []
    for output in outputs:
        for detect in output:
            scores = detect[5:]
            class_id = np.argmax(scores)
            conf = scores[class_id]
            if conf > 0.3:
                center_x = int(detect[0] * width)
                center_y = int(detect[1] * height)
                w = int(detect[2] * width)
                h = int(detect[3] * height)
                x = int(center_x - w/2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confs.append(float(conf))
                class_ids.append(class_id)
    return boxes, confs, class_ids


def draw_labels_video(boxes, confs, colors, class_ids, classes, img, current_frame, net, model_vgg, matrix, bird_image, rect_boundary, threshold): 
    indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    
    # drawing ROI on the frame
    cv2.line(img,rect_boundary[0],rect_boundary[1],(255,255,0),3)
    cv2.line(img,rect_boundary[0],rect_boundary[2],(255,255,0),3)
    cv2.line(img,rect_boundary[1],rect_boundary[3],(255,255,0),3)
    cv2.line(img,rect_boundary[2],rect_boundary[3],(255,255,0),3)
    
    current_person = 0
    person_pts = []
    warped_pts = []
    bb_pts = []
    violators_warp = []
    pers_violators = []
    
    # extracting bounding box and warped points for 'person' class
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            if label == 'person':
                
                bb_pts.append((x,y,w,h))
                
                X_mid = x+w//2
                mid_bottom_pt = (X_mid, y+h)
                person_pts.append(mid_bottom_pt)
                
                person_array = np.array([[[mid_bottom_pt[0], mid_bottom_pt[1]]]], dtype = 'float32')
                warped_coordinate = cv2.perspectiveTransform(person_array, matrix)[0][0]
                warped_pts.append((warped_coordinate[0], warped_coordinate[1]))
    
    
    # calculating social distancing violators
    for i in range(0, len(warped_pts)-1):
        for j in range(i+1, len(warped_pts)):
            
            x1, y1 = warped_pts[i]
            x2, y2 = warped_pts[j]
            dist = int(round(np.sqrt((x1 - x2)**2 + (y1 - y2)**2)))
            
            # if distance between the two points is less than allowed distance, then add them to voilators
            if dist < int(threshold):
                if warped_pts[i] not in violators_warp:
                    violators_warp.append(warped_pts[i])
                    pers_violators.append(person_pts[i])
                    
                if warped_pts[j] not in violators_warp:
                    violators_warp.append(warped_pts[j])
                    pers_violators.append(person_pts[j])
    
    
    # assigning colours to the violators and non-violators
    for i in range(0, len(warped_pts)):
        newImage = img.copy()
        x, y, w, h = bb_pts[i]
        axesLen = (w, h//2)
        angle = 115
        sub_person = newImage[y+1:y+h-1, x+1:x+w-1]
        
        if warped_pts[i] in violators_warp:

            cv2.rectangle(img, (x,y), (x+w, y+h), (0, 0, 255), 2)
            cv2.circle(img, person_pts[i], 5, (0,0, 255), -1)
                    
            cv2.circle(bird_image, warped_pts[i], 10, (0, 0, 255), -1)
            cv2.circle(bird_image, warped_pts[i], int(threshold), (0, 0, 255), 1)
            
        else:

            cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)
            cv2.circle(img, person_pts[i], 5, (0, 255, 0), -1)
                    
            cv2.circle(bird_image, warped_pts[i], 10, (0, 255, 0), -1)
            cv2.circle(bird_image, warped_pts[i], int(threshold), (0, 255, 0), 1)
    
    #stats
    total_people = len(warped_pts)
    total_violators = len(violators_warp)
    total_non_violators = total_people - total_violators
    
    # creating headings 
    org = (5, 35)
    font = cv2.FONT_HERSHEY_SIMPLEX 
    fontScale = 1
    color = (0, 0, 0)
    thickness = 2

    heading_normal = np.zeros((50, 1067, 3), np.uint8)
    heading_normal[:] = (145, 249, 255)
    heading_normal = cv2.putText(heading_normal, 'Normal View', org, font, fontScale, color, thickness, cv2.LINE_AA) 
    
    heading_bird = np.zeros((50, 400, 3), np.uint8)
    heading_bird[:] = (105, 186, 255)
    heading_bird = cv2.putText(heading_bird, "Bird's Eye View", org, font, fontScale, color, thickness, cv2.LINE_AA) 

    heading = cv2.hconcat([heading_normal, heading_bird])
    
    
    img = cv2.resize(img, (1067, 600))
    res = cv2.hconcat([img, bird_image])
    
    im_v = cv2.vconcat([heading, res])
    
    # stats 
    stats = np.zeros((150, 1467, 3), np.uint8)
    stats[:] = (255, 255, 255)
    stats = cv2.putText(stats, 'Stats (Social Distancing) :-', (10, 55), font, fontScale, color, thickness, cv2.LINE_AA) 
    total_str = 'Total people : ' + str(total_people)
    stats = cv2.putText(stats, total_str, (10, 120), font, fontScale, color, thickness, cv2.LINE_AA) 
    viol_str = 'Violators : ' + str(total_violators)
    stats = cv2.putText(stats, viol_str, (450, 120), font, fontScale, color, thickness, cv2.LINE_AA) 
    non_viol_str = 'Non-Violators : ' + str(total_non_violators)
    stats = cv2.putText(stats, non_viol_str, (890, 120), font, fontScale, color, thickness, cv2.LINE_AA)
    
    f_res = cv2.vconcat([im_v, stats])
    
    cv2.imshow("Output", f_res)
    
    return f_res


def get_mouse_points(event, x, y, flags, param):

    global mouseX, mouseY, mouse_pts
    if event == cv2.EVENT_LBUTTONDOWN:
        mouseX, mouseY = x, y
        cv2.circle(image, (x, y), 10, (0, 255, 255), 10)
        if "mouse_pts" not in globals():
            mouse_pts = []
        mouse_pts.append((x, y))
        print("Lenght of list : ", len(mouse_pts))
        print("Point detected")
        print(mouse_pts)


## Execution

In [8]:
#######
video_path = "test_video.mp4"

model, classes, colors, output_layers = load_yolo()
print("YOLO-v3 for Person Detection loaded")
print("------------------------------------------------")
net = load_yolo_face()
print("YOLO-v3 for Face Detection loaded")
print("------------------------------------------------")
    
"""
K.set_image_data_format('channels_first')
FRmodel = load_face_recog_model()
print("Face Recognition Model loaded")
print("------------------------------------------------")
database = load_database(FRmodel)
print("User Database loaded")
print("------------------------------------------------")
K.set_image_data_format('channels_last')
"""
    
model_vgg = load_VGG_SRGAN()
print("SRGAN loaded")
print("------------------------------------------------")

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
current_frame = 0

sz = (1467, 800)
out1 = cv2.VideoWriter("output/output_social_distancing.avi",cv2.VideoWriter_fourcc(*'XVID'), int(fps), sz)

f_no = 1
    
    
    
while cap.isOpened():
        
        
        
    ret, frame = cap.read()
        
    bird_image = np.zeros((600, 400, 3), np.uint8)
    bird_image[:] = (41, 41, 41)
        
    if ret == True:
        
        if f_no == 1:
                
            mouse_pts = []    
            cv2.namedWindow("image")
            cv2.setMouseCallback("image", get_mouse_points)
            num_mouse_points = 0
            
            while True:
                image = frame
                cv2.imshow("image", image)
                cv2.waitKey(1)
                    
                if len(mouse_pts) == 6:
                    f_no = f_no + 1
                    cv2.destroyWindow("image")
                    break
                four_points = mouse_pts
                
            pt1 = four_points[0:4]
            print("------ ROI defined ------")
            pt2 = [(0, 600), (400, 600), (0, 0), (400, 0)]
            
            pts1 = np.float32(np.array(pt1)) 
            pts2 = np.float32(np.array(pt2)) 
            
            matrix = cv2.getPerspectiveTransform(pts1, pts2) 
            
            ## threshold calculation
            t_pt1 = four_points[4]
            t_pt2 = four_points[5]
            
            thresh_pt1 = np.array([[[t_pt1[0], t_pt1[1]]]], dtype = 'float32')
            warped_t1 = cv2.perspectiveTransform(thresh_pt1, matrix)[0][0]
            
            thresh_pt2 = np.array([[[t_pt2[0], t_pt2[1]]]], dtype = 'float32')
            warped_t2 = cv2.perspectiveTransform(thresh_pt2, matrix)[0][0]
            
            threshold = round(np.sqrt((warped_t1[0] - warped_t2[0])**2 + (warped_t1[1] - warped_t2[1])**2))
            print("------ THRESHOLD defined ------")
        
        height, width, channels = frame.shape
        blob, outputs = detect_objects(frame, model, output_layers)
        boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
        image = draw_labels_video(boxes, confs, colors, class_ids, classes, frame, current_frame, net, model_vgg, matrix, bird_image, pt1, threshold)
        current_frame += 1        

        out1.write(image)
        #out2.write(bird_image)
        
        key = cv2.waitKey(9)
        if key == 27 or key == ord('q'):
            break
    else:
        print("**------- END OF VIDEO FILE -------**")
        break
            
cap.release()
cv2.destroyAllWindows()


YOLO-v3 for Person Detection loaded
------------------------------------------------
YOLO-v3 for Face Detection loaded
------------------------------------------------
SRGAN loaded
------------------------------------------------
Lenght of list :  1
Point detected
[(6, 441)]
Lenght of list :  2
Point detected
[(6, 441), (1049, 703)]
Lenght of list :  3
Point detected
[(6, 441), (1049, 703), (733, 4)]
Lenght of list :  4
Point detected
[(6, 441), (1049, 703), (733, 4), (1220, 20)]
Lenght of list :  5
Point detected
[(6, 441), (1049, 703), (733, 4), (1220, 20), (720, 374)]
Lenght of list :  6
Point detected
[(6, 441), (1049, 703), (733, 4), (1220, 20), (720, 374), (717, 202)]
------ ROI defined ------
------ THRESHOLD defined ------
**------- END OF VIDEO FILE -------**
